In [20]:
import pandas as pd
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegressionCV
from sklearn.neural_network import MLPClassifier
from sklearn.cross_validation import train_test_split,StratifiedKFold,KFold
import matplotlib.pyplot as plt
import numpy as np
from model_disply import *
from sklearn.externals import joblib

In [21]:
def model_lr(train_X,train_Y):
    lg = LogisticRegressionCV(class_weight='balanced')
    lg.fit(train_X,train_Y)  
    return lg

In [22]:
def model_NN(train_X,train_Y):
    nn = MLPClassifier(activation='logistic')
    nn.fit(train_X,train_Y)
    return nn

In [23]:
def model_CV(model_name):
    ks_list = []
#     weight = []
#     bias =[]
    for train,test in skf:
        train_data = samples.iloc[train]
        test_data = samples.iloc[test]
        model = model_name(train_data.iloc[:,:-1],train_data.iloc[:,-1])
        y_pro = model.predict_proba(test_data.iloc[:,:-1])
        y_pro = [x[1] for x in y_pro]
        ks = ks_value(test_data.iloc[:,-1],y_pro)        
        ks_list.append(ks)
#         weight.append(list(model.coef_[0]))
#         bias.append(list(model.intercept_))
    print ks_list
    return np.mean([x[0] for x in ks_list])

In [24]:
#save model
def save_model(model_name,model_s):
    count=0
    for train,test in skf:    
        train_data = samples.iloc[train]
        test_data = samples.iloc[test]
        model = model_name(train_data.iloc[:,:-1],train_data.iloc[:,-1])
        joblib.dump(model, "./out1/%s_%d.m"%(model_s,count))
        test_data.to_csv('./data/model_test_data_%d.csv'%count)
        count +=1

In [25]:
#save validation data
def save_valid_data():
    for i in xrange(5):
        train_X,val_X,train_Y,val_Y =train_test_split(samples.iloc[:,:-1],samples.iloc[:,-1],test_size=0.4)
        val_X['label']=val_Y
        val_X.to_csv('./out1/val_data_%d.csv'%i)

In [26]:
def valid_model(model_name,model_num):
    PSI =pd.DataFrame()
    model1= joblib.load('./out1/%s_%d.m'%(model_name,model_num))
    test_data = pd.read_csv('./data/model_test_data_%d.csv'%model_num,index_col=0)
    y_pro = model1.predict_proba(test_data.iloc[:,:-1])
    y_pro = [x[1] for x in y_pro]
    t = pd.cut(y_pro,[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).value_counts()
    PSI['t'] = t.apply(lambda x:x*1.0/t.sum() if x>0 else 0.00001)
    for i in xrange(5):
        val_data =pd.read_csv('./out1/val_data_%d.csv'%i,index_col=0)
        y_pre = model1.predict_proba(val_data.iloc[:,:-1])
        y_pre = [x[1] for x in y_pre]
        y_pre_bin = pd.cut(y_pre,[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).value_counts()
        PSI['t%d'%i] = y_pre_bin.apply(lambda x:x*1.0/y_pre_bin.sum() if x>0 else 0.00001)
        ks =ks_value(val_data.iloc[:,-1],y_pre)
        print ks
    return PSI

In [27]:
#cacilate psi
def compute_psi(PSI):
    psi_df =pd.DataFrame()
    for col in PSI.columns[1:]:
        PSI[col+'-t'] = PSI[col]-PSI['t']
        PSI[col+'ln_t'] = np.log(PSI[col]*1.0/PSI['t'])
        psi_df[col+'_psi']=PSI[col+'-t'] *PSI[col+'ln_t']
    return psi_df

In [28]:
#load data
def train_model():
    global samples,skf
    samples = pd.read_csv('./out1/samples_for_card_fillna_mean.csv',index_col=0) #card
#     samples = pd.read_csv('./out1/samples_41_fillna_mean.csv',index_col=0)
    samples = samples.drop(['交易额最大值','货币资金','历史_交易总量', '历史_交易额总量'],axis=1)
    skf = StratifiedKFold(samples.iloc[:,-1],n_folds=5)    
    save_valid_data()    
    lr_ks = model_CV(model_lr)
    print lr_ks
    print samples.head()
#     nn_ks = model_CV(model_NN)
#     print nn_ks

In [29]:
train_model()
samples.columns

[(0.357829220353789, 0.72330203101113788), (0.50386901424245822, 0.77526073791633954), (0.28787708870696427, 0.62795783335202426), (0.49994392732981952, 0.75356061455646517), (0.44802063474262643, 0.7607939890097567)]
0.419507977075
                     实收资本      注册地址      场地归属      注册时间    下游客户情况      涉诉信息  \
应收单号                                                                          
YS2016030800082  0.274318 -0.439962  0.393246  0.290983 -0.484329  0.070214   
YS2016030400009 -0.324096 -0.439962  0.393246 -0.164433 -0.086596 -0.093581   
YS2016030400024  0.274318  0.203259  0.393246  0.290983  1.757263  1.329492   
YS2016030700026 -0.324096  0.203259 -0.327458 -0.249272 -0.086596 -0.093581   
YS2016030700041  0.274318  0.203259  0.393246  0.290983  1.757263  1.329492   

                     有无贷款      企业征信       净资产      流动比率  ...    历史_交易额均值  \
应收单号                                                     ...               
YS2016030800082  0.079079 -1.742430  0.139249 -0.501472  ... 

Index([u'实收资本', u'注册地址', u'场地归属', u'注册时间', u'下游客户情况', u'涉诉信息', u'有无贷款',
       u'企业征信', u'净资产', u'流动比率', u'资产负债率', u'主营业务利润率', u'应收账款周转天数', u'存货周转天数',
       u'季度逾期占比', u'季度平均逾期天数', u'历史_逾期占比', u'历史_提前还款占比', u'季度提前还款占比',
       u'历史_平均逾期天数', u'平均逾期天数变化', u'逾期占比变化', u'订单总数', u'交易额总量', u'历史_交易稳定性',
       u'交易量最小值', u'交易总量', u'交易额最小值', u'交易额方差', u'交易量方差', u'提前还款占比变化',
       u'历史_账期订单交易量均值', u'历史_交易额均值', u'历史_账期订单交易额均值', u'交易额均值', u'交易稳定性',
       u'历史_最近连续交易月数', u'历史_账期合作时间（月）', u'订单时间间隔均值变化', u'账期订单交易量均值变化',
       u'账期订单交易额均值变化', u'label'],
      dtype='object')

In [30]:
save_model(model_lr,'model_lr')
P=pd.DataFrame()
psi_sum =pd.DataFrame()
for i in xrange(5):
    PSI = valid_model('model_lr',i)
    P = pd.concat([P,PSI])
    psi = compute_psi(PSI)
    psi_sum['lr_%d'%i] = psi.sum()
    
psi_sum

(0.53868038078564395, 0.82169821643505858)
(0.4649765439239123, 0.77843647580489694)
(0.44707752506725362, 0.77297627781853762)
(0.49333547454633048, 0.82218564316685416)
(0.4685961264908633, 0.79319400372031934)
(0.48044096728307256, 0.79155268628952835)
(0.42352676563202879, 0.74415204678362579)
(0.43695280019564681, 0.74942528735632175)
(0.50438948664418393, 0.79598254911407307)
(0.44925593609804138, 0.77330670751723374)
(0.50946493051756203, 0.80944304628515151)
(0.45623674571042988, 0.7587076666024033)
(0.42712154561017363, 0.76128637808755206)
(0.48201381082383171, 0.81008778973288376)
(0.47483313272786953, 0.79002079002079006)
(0.50815187657292915, 0.80003282634861583)
(0.41629715313925836, 0.76073195810037919)
(0.42377109317681577, 0.7678405478112007)
(0.48319147797227124, 0.81083721428189071)
(0.44714957872852612, 0.78922748659590758)
(0.51359557938505307, 0.80678958310537252)
(0.45324850588008486, 0.75983227299016776)
(0.45267791636096849, 0.76769381266813408)
(0.508243670039

,lr_0,lr_1,lr_2,lr_3,lr_4
t0_psi,0.082038,0.214402,0.223219,0.162954,0.337092
t1_psi,0.127748,0.226547,0.205913,0.192872,0.291612
t2_psi,0.146494,0.278985,0.242612,0.094266,0.171121
t3_psi,0.058798,0.214029,0.212020,0.144775,0.309916
t4_psi,0.064377,0.194701,0.244799,0.237909,0.359856


In [31]:
P
# samples

,t,t0,t1,t2,t3,t4
"(0.0, 0.1]",0.132616,0.127469,0.113106,0.132855,0.140036,0.152603
"(0.1, 0.2]",0.168459,0.166966,0.192101,0.199282,0.163375,0.159785
"(0.2, 0.3]",0.179211,0.152603,0.183124,0.154399,0.165171,0.156194
"(0.3, 0.4]",0.103943,0.105925,0.091562,0.077199,0.089767,0.078995
"(0.4, 0.5]",0.111111,0.100539,0.080790,0.091562,0.111311,0.098743
"(0.5, 0.6]",0.093190,0.078995,0.098743,0.091562,0.096948,0.104129
"(0.6, 0.7]",0.082437,0.113106,0.105925,0.118492,0.105925,0.104129
"(0.7, 0.8]",0.035842,0.091562,0.095153,0.096948,0.073609,0.078995
"(0.8, 0.9]",0.050179,0.032316,0.021544,0.023339,0.030521,0.034111
"(0.9, 1.0]",0.043011,0.030521,0.017953,0.014363,0.023339,0.032316


### 模型计分

In [14]:
def calc_score(p):
    odds = np.log(p/(1-p))
    A=441.5
    B=72.13
    score =A-B*odds
    return int(score)

In [15]:
#最优模型用于预测所有样本
model1 = joblib.load('./out1/model_lr_0.m')
pre = model1.predict_proba(samples.iloc[:,:-1])
result =pd.DataFrame(index=samples.index)
pre = [x[1] for x in pre]
result['lr_pre'] = list(pre)
result['score'] = result['lr_pre'].apply(lambda x:calc_score(x))
result['rank_score'] =result['score'].rank(method='max') /(1391/5.0)
result['rank_score'] =result['rank_score'].apply(lambda x:int(x) if x>0 and x<5 else 4)
result['label'] =samples['label']
print ks_value(samples.iloc[:,-1],pre)
result.head()

(0.47569267835631113, 0.7973229821978316)


,lr_pre,score,rank_score,label
应收单号,,,,
YS2016030800082,0.194267,544,3,1
YS2016030400009,0.632550,402,1,0
YS2016030400024,0.593701,414,1,0
YS2016030700026,0.244687,522,3,0
YS2016030700041,0.298434,503,2,0


In [16]:
ks_plot(samples.iloc[:,-1],pre,'all_samples_ks_6')
# auc_plot(samples.iloc[:,-1],pre,'all_samples_auc')

/home/sf/.local/lib/python2.7/site-packages/matplotlib/legend.py:326: UserWarning: Unrecognized location "mid right". Falling back on "best"; valid locations are
	right
	center left
	upper right
	lower right
	best
	center
	lower left
	center right
	upper left
	upper center
	lower center

  six.iterkeys(self.codes))))


KeyboardInterrupt: 

In [ ]:
#概率分布
pd.cut(pre,[0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1]).value_counts()

In [ ]:
#等量统计
tmp =pd.DataFrame()
tmp['等级界点'] = result.groupby(by='rank_score')['score'].apply(lambda x:(x.min(),x.max()))
tmp['样本数'] = result['rank_score'].value_counts()
tmp['违约样本数'] = result.groupby(by='rank_score')['label'].sum()
tmp

In [17]:
#等距统计
tmp =pd.DataFrame()
result['score_bin']= pd.cut(result['score'],[x for x in xrange(190,800,100)])
tmp ['样本数'] =result['score_bin'].value_counts()
tmp['违约样本数'] = result.groupby(by='score_bin')['label'].sum()
tmp

,样本数,违约样本数
"(490, 590]",477,17
"(390, 490]",462,79
"(290, 390]",200,66
"(590, 690]",186,3
"(190, 290]",40,21
"(690, 790]",26,0


### 评分卡计分

In [ ]:
wei_sf = pd.DataFrame(index=samples.iloc[:,:-1].columns)
wei_sf['weight'] =list(model1.coef_[0])
bias =model1.intercept_[0]
wei_sf =wei_sf.append(pd.DataFrame(index=['bias'],data=[bias],columns=['weight']))
print bias
wei_sf

In [ ]:
def calc_base(weight,sample):
    sf = weight*pd.Series(sample)
    base = sf.sum()
    return base   
def card_score(A,B,bias,base_score):
    score = A-B*(bias+base_score)
    return int(score)

In [ ]:
result['base_score']= samples.apply(lambda x:calc_base(wei_sf['weight'],x),axis=1)
result['card_score'] = result['base_score'].apply(lambda x:card_score(441.5,72.13,bias,x))
result.to_csv('./out1/model_pre_all_samples_score_6.csv')
result

In [ ]:
#对应客户和时间
feature_loan = pd.read_csv('./new_out/3m_loan_all.csv')
tmp = feature_loan[['客户','应收单号','起算日期']].merge(result['score'].reset_index(),on='应收单号',how='right')
tmp.to_csv('./out1/customer_score_times_6.csv',index=None)

In [ ]:
#对应客户旧评分卡分数
df =pd.DataFrame()
df['times'] = tmp.groupby(by='客户')['起算日期'].apply(list)
df['scores'] = tmp.groupby(by='客户')['score'].apply(list)
old_card = pd.read_excel('./data/feature_checked.xlsx')
old_card.index = [x.encode('utf-8') for x in old_card.index]
df =df.merge(old_card[u'评分'].reset_index(),left_index=True,right_on='index',how='left')
df.to_csv('./out1/customer_score_times_6.csv',encoding='utf-8')
df

### 评分卡制作

In [ ]:
card =pd.read_csv('./out1/woes_cols_2.csv')
card = card.merge(wei_sf.reset_index(),left_on='变量',right_on='index',how='right')
# card['base'] = card['woe']*card['weight']*(-72.13)
card.to_csv('./out1/card_6.csv')    
card